<a href="https://colab.research.google.com/github/jamelof23/ASL/blob/main/Insider.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import time

def get_sp500_symbols():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)

    if response.status_code == 200:
        table = pd.read_html(response.content, match='Symbol')[0]
        return table['Symbol'].tolist()
    else:
        print("Failed to retrieve S&P 500 data.")
        return []

def get_insider_trades_finnhub(api_key, symbol):
    url = f'https://finnhub.io/api/v1/stock/insider-transactions?symbol={symbol}&token={api_key}'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if 'data' in data and data['data']:
            df = pd.DataFrame(data['data'])
            buy_transactions = df[df['change'] > 0]
            sell_transactions = df[df['change'] < 0]
            return {'Buy': buy_transactions, 'Sell': sell_transactions}
    else:
        print(f"Failed to retrieve data for {symbol}. Status code: {response.status_code}")
    return None

# Get your API key from finnhub.io
api_key = 'csea3f1r01qs1ihohl6gcsea3f1r01qs1ihohl70'

# Fetch the list of S&P 500 symbols
sp500_symbols = get_sp500_symbols()

# Dictionary to store insider trades
all_insider_trades = {}

for symbol in sp500_symbols[:5]:  # Limited to 5 for demonstration
    print(f"Fetching insider transactions for {symbol}...")
    trades = get_insider_trades_finnhub(api_key, symbol)
    if trades:
        all_insider_trades[symbol] = trades
    time.sleep(1)

# Display results
for symbol, trades in all_insider_trades.items():
    print(f"\n{symbol} Insider Buy Transactions:")
    print(trades['Buy'] if not trades['Buy'].empty else "No recent buy transactions.")
    print(f"\n{symbol} Insider Sell Transactions:")
    print(trades['Sell'] if not trades['Sell'].empty else "No recent sell transactions.")
